In [10]:
import os
import pandas as pd
import numpy as np
os.chdir('/Users/younghun/Desktop/gitrepo/data/movie_dataset/')

In [42]:
import sklearn
import random 

# Pyspark Library #
# SQL
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import mean, col, split, regexp_extract, when, lit
# ML
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, IndexToString
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

### Spark Session

In [4]:
# 스파크 세션 만들기
spark = SparkSession\
        .builder\
        .appName('recommender_system')\
        .getOrCreate()

- ``Pyspark.toPandas()``메소드는 Spark의 DataFrame 자료구조를 Pandas의 테이블 모습으로 보여주도록 해줌!
<br><br>
- ``inferSchema=True``: 해당 csv파일 내부에 존재하는 헤더(스키마)를 알아서 캐치하여 반환
- ``header=True``: 헤더(스키마)가 있다고 알려주기

In [17]:
df = spark.read.csv(os.getcwd() + '/movie_ratings_df.csv',
                   inferSchema=True, header=True)

- ``limit(), select(), show()`` => 데이터 미리보기

In [18]:
df.limit(3).toPandas()

,userId,title,rating
0,1,Three Colors: Red,1.0
1,11,Three Colors: Red,3.5
2,22,Three Colors: Red,5.0


- 유저에 대한 정보가 주어졌을 때, 유저가 볼 만한 영화를 추천해주자!
- ``printSchema()``로 테이블의 스키마 살펴보기

In [19]:
df.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- rating: double (nullable = true)



- title 변수명이 string으로 되어 있기 때문에 MLLib을 이용해서 수치형 값으로 convert하기
- ``StringIndexer(inputCol='변환할 칼럼명', outputCol='변환후 생성할 칼럼명')`` => 이 객체를 ``fit, transform``메소드에 dataframe 넣어주기


### Convert string to numeric value

In [39]:
stringIndexer = StringIndexer(inputCol='title',
                             outputCol='title_new')
model = stringIndexer.fit(df)
indexed = model.transform(df)
indexed.limit(5).toPandas()

,userId,title,rating,title_new
0,1,Three Colors: Red,1.0,6.0
1,11,Three Colors: Red,3.5,6.0
2,22,Three Colors: Red,5.0,6.0
3,24,Three Colors: Red,5.0,6.0
4,29,Three Colors: Red,3.0,6.0


### Build recommender-system - ALS(Alternate Least Squares)

- ``regParam``: 데이터셋의 크기에 의존하지 않고 추천 시스템의 일반화를 시켜주기 위한 정규화 항
- ``coldStartStrategy``: 아직 평가되지 않은 즉, 결측치(NaN)값을 갖는 row들을 제외하고 모델 성능이 평가됨(``drop``)
    * Train 데이터에는 있지만 Test에는 존재하지 않는 즉, 새로운 유저나 제품이 출시되었을 때, 이에 대한 히스토리가 없어서 평점이 없는 Cold Start문제가 발생하는 것에 대한 대응 방법
    * ``nan``방법도 있는데, 이는 결측치를 포함해서 모델 성능 평가
- ``nonnegative=True``: least squares 할 때 음수(-)값이 미포함되어 있는 제한사항을 줄 것인지 -> **평점과 영화제목, 유저아이디는 모두 음수값이 없기** 때문에 ``True``로!

In [24]:
train, test = indexed.randomSplit([0.75, 0.25])
# ALS recommender algorithm
from pyspark.ml.recommendation import ALS

rec = ALS(maxIter=10,
         regParam=0.01,
         userCol='userId',
         itemCol='title_new',
         ratingCol='rating', # label -> predict할 때는 필요 없음!
         nonnegative=True,
         coldStartStrategy='drop')
# ALS모델 학습 -> dataframe을 넣어주기
rec_model = rec.fit(train)

# transform을 이용해 예측 -> dataframe을 넣어주기
pred_ratings = rec_model.transform(test)
pred_ratings.limit(5).toPandas()

Exception ignored in: <function JavaWrapper.__del__ at 0x7fadb20aa830>
Traceback (most recent call last):
  File "/usr/local/Cellar/apache-spark/3.0.1/libexec/python/pyspark/ml/wrapper.py", line 42, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'ALS' object has no attribute '_java_obj'


,userId,title,rating,title_new,prediction
0,1829,Yesterday,2.5,148.0,2.160393
1,6466,Yesterday,4.5,148.0,3.411708
2,15447,Yesterday,3.0,148.0,3.053997
3,23364,Yesterday,4.0,148.0,3.635531
4,25591,Yesterday,5.0,148.0,2.422732


In [25]:
# Get metric for training
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(labelCol='rating',
                               predictionCol='prediction',
                               metricName='rmse')
# evaluate 메소드에 예측값 담겨있는 dataframe 넣어주기
rmse = evaluator.evaluate(pred_ratings)
print("RMSE:", rmse)

RMSE: 0.9037694696632295


In [28]:
mae_eval = RegressionEvaluator(labelCol='rating',
                              predictionCol='prediction',
                              metricName='mae')
mae = mae_eval.evaluate(pred_ratings)
print("MAE:", mae)

MAE: 0.6877014610850091


- rating에 대한 정답과 예측차이의 평균 ``RMSE``값이 0.9이다.
- rating에 대한 정답과 예측차이의 평균 ``MAE``값이 0.68이다.

### 특정 사용자가 좋아할만한 영화 추천해주기

In [31]:
# 숫자로 바꾼 영화제목들 중 Unique한 값들만 담아 추출하기 -> Dataframe 반환
unique_movies = indexed.select("title_new").distinct()
print(unique_movies.show(5), type(unique_movies))

+---------+
|title_new|
+---------+
|    305.0|
|    596.0|
|    769.0|
|    496.0|
|    299.0|
+---------+
only showing top 5 rows

None <class 'pyspark.sql.dataframe.DataFrame'>


- Pyspark에서 Column 이름을 rename하는 방법들
    * ``withColumnRenamed``: ``df.withColumnRenamed('변경 전', '변경 후')
    * ``toDF``: 한 번에 칼럼명을 바꾸는 방법. 순차적으로 입력.이 방법은 다른 방법보다 매우 느림.``df.toDF('바꿀변수명1', '바꿀변수명2', ...)``
    * ``alias``: ``df.select(col('기존 변수명').alias('변경 후 변수명'))``

- Pyspark에서 ``filter``의 역할(``where``과 기능 동일)
    * ``df.filter(condition)``
        * condition에 Pyspark문법으로 구성된 조건을 넣어도 되고 SQL문도 넣어도 수행됨
        * 2개 이상의 condition 넣어줄 때는 ``논리 연산자(&, |)`` 사용 가능
        * ``array_contains(df.column, 'value')``을 넣어주어 Array에서 특정 value를 갖는 row들 추출 가능
        * nested(중첩)된 변수들 일때도 사용 가능 ex)name 칼럼안에 lastname 칼럼이 있다 => ``df.name.lastname == 'John'`` 식으로 가능!
        

- Pyspark에서 ``lit()``의 역할
    * 데이터프레임에 문자열이나 상수값을 할당하면서 새로운 칼럼을 만들기 위함
    * 칼럼 type으로 반환
    * ``lit(value).alias('변수명')``: value값을 동일하게 할당하면서 만들어 '변수명'으로 칼럼이름 지정
    * ``df2.withColumn('lit_value2', when(col('Salary') >=400 & col('Salary') <= 500, lit('100')).otherwise(lit('200'))``: lit_value2라는 새로운 변수를 만드는데, 400 <= Salary 변수값 <= 500일때는 lit_value2변수에 100값을, 그렇지 않으면 200값을 넣어라!
<br><br>
- Pyspark에서 ``typedLit()``의 역할
    * ``lit()``과는 다르게 **Array, Dict**처럼 collection type을 다룰 수 있음
 

In [43]:
def top_movies(user_id, n):
    """
    특정 user_id가 좋아할 만한 n개의 영화 추천해주는 함수
    """
    # unique_movies 데이터프레임을 'a'라는 데이터프레임으로 alias시키기
    a = unique_movies.alias('a')
    
    # 특정 user_id가 본 영화들만 담은 새로운 데이터프레임 생성
    watched_movies = indexed.filter(indexed['userId'] == user_id)\
                            .select('title_new')
    
    # 특정 user_id가 본 영화들을 'b'라는 데이터프레임으로 alias시키기
    b = watched_movies.alias('b')
    
    # unique_movies를 기준으로 watched_movies를 조인시켜서 user_id가 보지 못한 영화들 파악 가능
    total_movies = a.join(b, a['title_new'] == b['title_new'],
                         how='left')
    
    # b 데이터프레임의 title_new값이 결측치를 갖고 있는 행의 a.title_new를 뽑아냄으로써 user_id가 아직 못본 영화들 추출
    # col('b.title_new') => b 데이터프레임의 title_new칼럼 의미(SQL처럼 가능!)
    remaining_movies = total_movies\
                       .where(col('b.title_new').isNull())\
                       .select('a.title_new').distinct()
    # remaining_movies 데이터프레임에 특정 user_id값을 동일하게 새로운 변수로 추가해주기
    remaining_movies = remaining_movies.withColumn('userId',
                                                  lit(int(user_id)))
    # 위에서 만든 ALS 모델을 사용하여 추천 평점 예측 후 n개 만큼 view -> 
    recommender = rec_model.transform(remaining_movies)\
                           .orderBy('prediction', ascending=False)\
                           .limit(n)
    # StringIndexer로 만든 것을 역으로 바꾸기 위해 IndexToString사용(영화제목을 숫자->한글제목)
    movie_title = IndexToString(inputCol='title_new',
                               outputCol='title',
                               labels=model.labels) #여기서 model.labels는 StringIndexer에서 fit시켰을 때 생긴 레이블. 즉, 영화 제목들
    # transform해서 영화제목을 숫자->한글로 변환! => dataframe으로 반환
    final_recommendations = movie_title.transform(recommender)
    
    return final_recommendations.show(n, truncate=False)


In [44]:
top_movies(1829, 5)

+---------+------+----------+------------------------------+
|title_new|userId|prediction|title                         |
+---------+------+----------+------------------------------+
|6725.0   |1829  |17.840975 |Some Kind of a Nut            |
|6852.0   |1829  |15.735489 |Way Beyond Weight             |
|6227.0   |1829  |12.143332 |Exterminators of the Year 3000|
|5821.0   |1829  |10.902253 |Insidious: Chapter 2          |
|6016.0   |1829  |10.588577 |I Can Do Bad All By Myself    |
+---------+------+----------+------------------------------+

